In [1]:
import sys
import os
import numpy as np
sys.path.extend(['/home/bastien/Documents/work/mermoz', 
                 '/home/bastien/Documents/work/mermoz/src',
                 '/home/bastien/Documents/work/mdisplay',
                 '/home/bastien/Documents/work/mdisplay/src'])
from mermoz.wind import DiscreteWind
from mermoz.mdf_manager import MDFmanager
from mermoz.params_summary import ParamsSummary
from mermoz.misc import *

from mdisplay.geodata import GeoData

wind_data_dir = '/home/bastien/Documents/data/wind/ncdc'
interm_dir = os.path.join(wind_data_dir, 'test.mz')
output_dir = os.path.join(wind_data_dir, 'san-juan-dublin-flattened-ortho.mz')
for d in [interm_dir, output_dir]:
    if not os.path.exists(d):
        os.mkdir(d)

gribfile = 'gfs_4_20220301_1200_000.grb2'
gd = GeoData()
bl = gd.get_coords('San Juan', units='deg')
tr = gd.get_coords('Dublin', units='deg')
bl, tr = enlarge(bl, tr)

In [2]:
mdfm = MDFmanager()
mdfm.set_output_dir(interm_dir)
mdfm.dump_wind_from_grib2(os.path.join(wind_data_dir, gribfile), bl, tr)
wind = DiscreteWind(interp='linear')
wind.load(os.path.join(interm_dir, 'wind.h5'))
print(wind.x_min, wind.x_max, wind.y_min, wind.y_max)
ps = ParamsSummary({}, interm_dir)
ps.add_param('coords', 'gcs')
ps.add_param('bl_wind', list(RAD_TO_DEG * wind.grid[0, 0]))
ps.add_param('tr_wind', list(RAD_TO_DEG * wind.grid[-1, -1]))
ps.add_param('nx_wind', wind.grid.shape[0])
ps.add_param('ny_wind', wind.grid.shape[1])
ps.dump()

nx : 132, ny : 77
Loading wind values...        Done
-1.2042771838760873 -0.061086523819801536 0.29670597283903605 0.9599310885968813


In [3]:
lon_0 = (bl[0] + tr[0])*0.5
lat_0 = (bl[1] + tr[1])*0.5
wind.flatten(proj='ortho', lon_0=lon_0, lat_0=lat_0)

-3305149.920136907 3294131.906158604 -2066763.525940521 2428553.6469813613


In [ ]:
wind.unstructured

In [4]:
mdfm = MDFmanager()
mdfm.set_output_dir(output_dir)
mdfm.dump_wind(wind)

In [5]:
ps = ParamsSummary({}, output_dir)
ps.add_param('coords', 'cartesian')
ps.add_param('bl_wind', (wind.grid[:, :, 0].min(), wind.grid[:, :, 1].min()))
ps.add_param('tr_wind', (wind.grid[:, :, 0].max(), wind.grid[:, :, 1].max()))
ps.add_param('nx_wind', wind.grid.shape[0])
ps.add_param('ny_wind', wind.grid.shape[1])
ps.dump()

{"coords": "gcs", "bl_wind": [-38.3, -8.8], "tr_wind": [-14.800000000000002, 17.7], "nx_wind": 48, "ny_wind": 54, "date_wind": 1648112400000.0, "point_init": [-17.447938, 14.693425], "max_time": 108000.0, "geodesic_time": 130795.63554977896, "airspeed": 23.0, "point_target": [-35.2080905, -5.805398], "target_radius": 55659.74539663678, "nt_pmp": 1000, "pmp_time": 0.30774879455566406}